# Kapitola 34: Teachable Machine - AI se uci poslouchat

V teto kapitole naucime AI rozpoznavat zvuky! Pouzijeme Google Teachable Machine pro vytvoreni zvukoveho klasifikatoru a naucime se zaklady prace s audiem v Pythonu.

## Co se naucime:
- Jak funguje klasifikace zvuku
- Prace s Teachable Machine pro audio
- Zaklady zvukovych signalu v Pythonu
- Spektrogramy - jak AI "vidi" zvuk
- MFCC - priznaky pro rozpoznavani zvuku

---

## 1. Instalace knihoven

Pro praci se zvukem potrebujeme specialni knihovny:

In [ ]:
# Instalace knihoven pro praci se zvukem
!pip install numpy matplotlib librosa scipy soundfile

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Nastaveni pro grafy
plt.rcParams['figure.figsize'] = [12, 6]
print("Knihovny uspesne nacteny!")
print()
print("V teto kapitole se naucime:")
print("1. Jak AI 'slysi' zvuky")
print("2. Jak pouzit Teachable Machine pro audio")
print("3. Jak pracovat se zvukem v Pythonu")

---

## 2. Jak AI "slysi" zvuk?

Zvuk je vlneni vzduchu. Mikrofon ho prevadi na elektricky signal, ktery pocitac reprezentuje jako **posloupnost cisel** (vzorky).

### Zakladni pojmy:
- **Vzorkovaci frekvence (Sample Rate)**: Kolikrat za sekundu merime zvuk (napr. 44100 Hz = 44100 vzorku/s)
- **Amplituda**: Hlasitost zvuku
- **Frekvence**: Jak rychle se vlna opakuje (Hz) - urcuje vysku tonu

In [ ]:
# Vytvorime si jednoduchy zvukovy signal - sinusovku

# Parametry
vzorkovaci_frekvence = 44100  # Hz (CD kvalita)
delka = 0.1  # sekund
frekvence_tonu = 440  # Hz (nota A4)

# Vytvoreni casove osy
t = np.linspace(0, delka, int(vzorkovaci_frekvence * delka))

# Vytvoreni sinusoveho signalu
signal = np.sin(2 * np.pi * frekvence_tonu * t)

# Vizualizace
plt.figure(figsize=(14, 4))
plt.plot(t * 1000, signal, 'b-', linewidth=0.5)
plt.xlabel('Cas (ms)', fontsize=12)
plt.ylabel('Amplituda', fontsize=12)
plt.title(f'Zvukovy signal - sinusovka 440 Hz (nota A4)', fontsize=14)
plt.grid(True, alpha=0.3)
plt.xlim(0, 10)  # Zobrazime jen prvnich 10 ms
plt.tight_layout()
plt.show()

print(f"Vzorkovaci frekvence: {vzorkovaci_frekvence} Hz")
print(f"Pocet vzorku: {len(signal)}")
print(f"Frekvence tonu: {frekvence_tonu} Hz")

In [ ]:
# Porovnani ruznych zvuku - nizky vs vysoky ton

frekvence_nizka = 200   # Hz - nizky ton
frekvence_vysoka = 800  # Hz - vysoky ton

signal_nizky = np.sin(2 * np.pi * frekvence_nizka * t)
signal_vysoky = np.sin(2 * np.pi * frekvence_vysoka * t)

fig, axes = plt.subplots(2, 1, figsize=(14, 6))

axes[0].plot(t * 1000, signal_nizky, 'b-', linewidth=1)
axes[0].set_title(f'Nizky ton: {frekvence_nizka} Hz', fontsize=12)
axes[0].set_xlim(0, 20)
axes[0].set_ylabel('Amplituda')
axes[0].grid(True, alpha=0.3)

axes[1].plot(t * 1000, signal_vysoky, 'r-', linewidth=1)
axes[1].set_title(f'Vysoky ton: {frekvence_vysoka} Hz', fontsize=12)
axes[1].set_xlim(0, 20)
axes[1].set_xlabel('Cas (ms)')
axes[1].set_ylabel('Amplituda')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("Nizky ton ma mensi frekvenci = delsi vlny")
print("Vysoky ton ma vetsi frekvenci = kratsi vlny")

---

## 3. Google Teachable Machine pro zvuk

Teachable Machine umoznuje trenovat AI model BEZ PSANI KODU!

### Jak vytvorit zvukovy klasifikator:

**Krok 1: Otevrete Teachable Machine**
- Jdete na: https://teachablemachine.withgoogle.com/
- Kliknete na "Get Started"
- Vyberte **"Audio Project"**

**Krok 2: Nastavte tridy (kategorie zvuku)**

Pro nas projekt "Tleskni, luskni, zapiskej":

| Trida | Popis | Pocet vzorku |
|-------|-------|-------------|
| Background Noise | Ticho, sum pozadi | 20 sekund |
| Tlesknuti | Zvuk tleskani | 8-10 vzorku |
| Lusknuti | Zvuk lusknuti prsty | 8-10 vzorku |
| Piskani | Zvuk hvizdu | 8-10 vzorku |

In [ ]:
# Vizualizace procesu treninku v Teachable Machine

fig, ax = plt.subplots(figsize=(14, 8))

# Kroky procesu
kroky = [
    ('1. Nahrat\npozadi', 'Nahrajte 20 sekund ticha\n(model se nauci ignorovat sum)', 'lightblue'),
    ('2. Nahrat\ntlesknuti', 'Nahrajte 8-10 vzorku\ntlesknuti (po 2 sekundach)', 'lightgreen'),
    ('3. Nahrat\nlusknuti', 'Nahrajte 8-10 vzorku\nlusknuti prsty', 'lightyellow'),
    ('4. Nahrat\npiskani', 'Nahrajte 8-10 vzorku\nhvizdu/piskani', 'lightsalmon'),
    ('5. Trenovat\nmodel', 'Kliknete na "Train Model"\na pockejte', 'lightpink'),
    ('6. Testovat\nv realnem case', 'Zkuste tlesnout, lusknout\na sledujte predikce!', 'lavender')
]

for i, (nazev, popis, barva) in enumerate(kroky):
    x = i * 2.2
    rect = plt.Rectangle((x, 0), 2, 4, facecolor=barva, edgecolor='black', linewidth=2)
    ax.add_patch(rect)
    ax.text(x + 1, 4.3, nazev, ha='center', va='bottom', fontsize=10, fontweight='bold')
    ax.text(x + 1, 2, popis, ha='center', va='center', fontsize=8)
    
    if i < len(kroky) - 1:
        ax.annotate('', xy=(x + 2.2, 2), xytext=(x + 2, 2),
                   arrowprops=dict(arrowstyle='->', color='gray', lw=2))

ax.set_xlim(-0.5, 13.5)
ax.set_ylim(-0.5, 5.5)
ax.set_aspect('equal')
ax.axis('off')
ax.set_title('Proces treninku zvukoveho klasifikatoru v Teachable Machine', 
             fontsize=14, fontweight='bold', pad=20)

plt.tight_layout()
plt.show()

### DULEZITE: Proc nahravame pozadi?

Trida "Background Noise" je **klicova**! Uci model, co ma **ignorovat**.

Bez ni by model reagoval na kazdy sum jako na tlesknuti nebo lusknuti.

In [ ]:
# Demonstrace - proc je pozadi dulezite

print("=" * 60)
print("PROC JE TRIDA 'POZADI' TAK DULEZITA?")
print("=" * 60)
print()
print("BEZ tridy pozadi:")
print("  - Model vidi jen: tlesknuti, lusknuti, piskani")
print("  - Kazdy zvuk MUSI byt jeden z nich")
print("  - Sum ventilace? -> 'To je asi tlesknuti!'")
print("  - Skripnuti zidle? -> 'To je lusknuti!'")
print()
print("S tridou pozadi:")
print("  - Model vidi: pozadi, tlesknuti, lusknuti, piskani")
print("  - Muze rict: 'Toto je jen sum pozadi, nic nedela'")
print("  - Sum ventilace? -> 'To je pozadi (ignoruj)'")
print("  - Skutecne tlesknuti? -> 'To je tlesknuti!'")
print()
print("TIP: Nahrajte pozadi v prostredi, kde budete model pouzivat!")

---

## 4. Jak AI "vidi" zvuk - Spektrogramy

Neuronove site pro obrazky umi zpracovat 2D data. Jak prevedeme 1D zvuk na 2D obrazek?

Odpoved: **SPEKTROGRAM**!

Spektrogram ukazuje:
- **Osa X**: Cas
- **Osa Y**: Frekvence
- **Barva**: Intenzita (hlasitost) dane frekvence v danem case

In [ ]:
# Vytvoreni slozitejsiho signalu a jeho spektrogramu
from scipy import signal as scipy_signal

# Parametry
fs = 8000  # Vzorkovaci frekvence
delka = 1.0  # sekunda
t = np.linspace(0, delka, int(fs * delka))

# Signal s meniciimi se frekvencemi (chirp - "cvrkot")
# Frekvence roste od 100 Hz do 2000 Hz
chirp_signal = scipy_signal.chirp(t, f0=100, f1=2000, t1=delka, method='linear')

# Vypocet spektrogramu
frekvence, casy, spektrogram = scipy_signal.spectrogram(chirp_signal, fs, nperseg=256)

# Vizualizace
fig, axes = plt.subplots(2, 1, figsize=(14, 8))

# Casovy prubeh
axes[0].plot(t, chirp_signal, 'b-', linewidth=0.5)
axes[0].set_xlabel('Cas (s)', fontsize=12)
axes[0].set_ylabel('Amplituda', fontsize=12)
axes[0].set_title('Casovy prubeh signalu (chirp - frekvence roste)', fontsize=14)
axes[0].grid(True, alpha=0.3)

# Spektrogram
im = axes[1].pcolormesh(casy, frekvence, 10 * np.log10(spektrogram + 1e-10), 
                        shading='gouraud', cmap='viridis')
axes[1].set_xlabel('Cas (s)', fontsize=12)
axes[1].set_ylabel('Frekvence (Hz)', fontsize=12)
axes[1].set_title('Spektrogram - "obrazek" zvuku pro AI', fontsize=14)
plt.colorbar(im, ax=axes[1], label='Intenzita (dB)')

plt.tight_layout()
plt.show()

print("Spektrogram ukazuje, jak se frekvence meni v case.")
print("Tento 'obrazek' zvuku muze AI analyzovat jako bezny obrazek!")

In [ ]:
# Porovnani spektrogramu ruznych typu zvuku

np.random.seed(42)
fs = 8000
delka = 0.5
t = np.linspace(0, delka, int(fs * delka))

# Ruzne typy zvuku
zvuky = {
    'Cisty ton (440 Hz)': np.sin(2 * np.pi * 440 * t),
    'Akord (3 tony)': (np.sin(2 * np.pi * 262 * t) + 
                       np.sin(2 * np.pi * 330 * t) + 
                       np.sin(2 * np.pi * 392 * t)) / 3,
    'Sum (nahodny)': np.random.randn(len(t)) * 0.5,
    'Impulz (tlesknuti)': np.exp(-50 * t) * np.sin(2 * np.pi * 200 * t) * (t < 0.1)
}

fig, axes = plt.subplots(2, 4, figsize=(16, 8))

for i, (nazev, signal) in enumerate(zvuky.items()):
    # Casovy prubeh
    axes[0, i].plot(t[:1000], signal[:1000], 'b-', linewidth=0.5)
    axes[0, i].set_title(nazev, fontsize=11)
    axes[0, i].set_xlabel('Cas (s)')
    if i == 0:
        axes[0, i].set_ylabel('Amplituda')
    
    # Spektrogram
    f, cas, Sxx = scipy_signal.spectrogram(signal, fs, nperseg=128)
    axes[1, i].pcolormesh(cas, f, 10 * np.log10(Sxx + 1e-10), 
                          shading='gouraud', cmap='viridis')
    axes[1, i].set_xlabel('Cas (s)')
    if i == 0:
        axes[1, i].set_ylabel('Frekvence (Hz)')

plt.suptitle('Ruzne zvuky a jejich spektrogramy', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print("Kazdy typ zvuku ma charakteristicky 'otisk' ve spektrogramu!")
print("AI se uci rozpoznavat tyto vzory.")

---

## 5. MFCC - Jak profesionalne zpracovat zvuk

**MFCC (Mel-Frequency Cepstral Coefficients)** jsou nejpouzivanejsi priznaky pro rozpoznavani zvuku a reci.

Proc MFCC?
- Zachycuji to, co je pro lidske ucho dulezite
- Kompaktni reprezentace (mene dat)
- Funguje skvele pro klasifikaci

In [ ]:
# Demonstrace MFCC pomoci jednoducheho vypoctu
# (Pouzijeme zjednoduseny priklad)

print("=" * 60)
print("MFCC - Mel-Frequency Cepstral Coefficients")
print("=" * 60)
print()
print("Proces vytvoreni MFCC:")
print()
print("1. ROZDELENI NA RAMCE")
print("   - Zvuk se rozdeli na kratke useky (20-40 ms)")
print("   - Kazdy usek se zpracuje samostatne")
print()
print("2. FOURIEROVA TRANSFORMACE (FFT)")
print("   - Prevede signal z casove do frekvencni domeny")
print("   - Ziskame spektrum - jake frekvence jsou pritomny")
print()
print("3. MEL FILTROVA BANKA")
print("   - Aplikuje se sada filtru podle Mel skaly")
print("   - Mel skala odpovida vnimani frekvence lidskym uchem")
print("   - Nizke frekvence jsou rozliseny lepe nez vysoke")
print()
print("4. LOGARITMUS")
print("   - Lidske ucho vnima hlasitost logaritmicky")
print()
print("5. DCT (Discrete Cosine Transform)")
print("   - Vysledkem je 12-20 MFCC koeficientu")
print("   - Kompaktni reprezentace zvuku")

In [ ]:
# Vizualizace Mel skaly vs linerani frekvence

def hz_to_mel(hz):
    """Prevod z Hz na Mel."""
    return 2595 * np.log10(1 + hz / 700)

def mel_to_hz(mel):
    """Prevod z Mel na Hz."""
    return 700 * (10**(mel / 2595) - 1)

frekvence_hz = np.linspace(0, 8000, 1000)
frekvence_mel = hz_to_mel(frekvence_hz)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Hz vs Mel
axes[0].plot(frekvence_hz, frekvence_mel, 'b-', linewidth=2)
axes[0].set_xlabel('Frekvence (Hz)', fontsize=12)
axes[0].set_ylabel('Mel', fontsize=12)
axes[0].set_title('Prevod Hz na Mel skalu', fontsize=14)
axes[0].grid(True, alpha=0.3)

# Vizualizace Mel filtru
pocet_filtru = 10
mel_min = hz_to_mel(0)
mel_max = hz_to_mel(4000)
mel_points = np.linspace(mel_min, mel_max, pocet_filtru + 2)
hz_points = mel_to_hz(mel_points)

for i in range(pocet_filtru):
    # Trojuhelnikovy filtr
    filtr = np.zeros(len(frekvence_hz))
    start = hz_points[i]
    center = hz_points[i + 1]
    end = hz_points[i + 2]
    
    for j, f in enumerate(frekvence_hz):
        if start <= f < center:
            filtr[j] = (f - start) / (center - start)
        elif center <= f < end:
            filtr[j] = (end - f) / (end - center)
    
    axes[1].plot(frekvence_hz, filtr, linewidth=1.5)

axes[1].set_xlabel('Frekvence (Hz)', fontsize=12)
axes[1].set_ylabel('Vaha filtru', fontsize=12)
axes[1].set_title(f'Mel filtrova banka ({pocet_filtru} filtru)', fontsize=14)
axes[1].set_xlim(0, 4000)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("Mel skala:")
print("- Nizke frekvence jsou rozliseny jemneji")
print("- Vysoke frekvence jsou sdruzeny do sirsich pasem")
print("- Odpovida tomu, jak lidske ucho vnima zvuk")

---

## 6. Simulace klasifikace zvuku

Ukazeme si, jak by vypadal jednoduchy klasifikator zvuku:

In [ ]:
# Simulace klasifikace zvuku

class JednoduchyZvukovyKlasifikator:
    """Simulace klasifikatoru zvuku (zjednodusena verze)."""
    
    def __init__(self):
        self.tridy = ['pozadi', 'tlesknuti', 'lusknuti', 'piskani']
        # Simulovane charakteristiky zvuku
        self.charakteristiky = {
            'pozadi': {'energie': (0, 0.1), 'peak_freq': (0, 500)},
            'tlesknuti': {'energie': (0.5, 1.0), 'peak_freq': (100, 500)},
            'lusknuti': {'energie': (0.3, 0.7), 'peak_freq': (500, 2000)},
            'piskani': {'energie': (0.2, 0.5), 'peak_freq': (1000, 3000)}
        }
    
    def extrahuj_priznaky(self, signal):
        """Extrahuje jednoduche priznaky ze signalu."""
        energie = np.mean(signal**2)
        # Simulovana peak frekvence
        fft = np.abs(np.fft.fft(signal))
        frekvence = np.fft.fftfreq(len(signal), 1/8000)
        peak_idx = np.argmax(fft[:len(fft)//2])
        peak_freq = abs(frekvence[peak_idx])
        return {'energie': energie, 'peak_freq': peak_freq}
    
    def predikuj(self, signal):
        """Predikuje tridu zvuku."""
        priznaky = self.extrahuj_priznaky(signal)
        
        # Vypocet pravdepodobnosti pro kazdou tridu
        pravdepodobnosti = {}
        for trida, char in self.charakteristiky.items():
            # Jednoducha metrika - jak dobre priznaky odpovidaji
            e_min, e_max = char['energie']
            f_min, f_max = char['peak_freq']
            
            e_score = 1 if e_min <= priznaky['energie'] <= e_max else 0.1
            f_score = 1 if f_min <= priznaky['peak_freq'] <= f_max else 0.1
            
            pravdepodobnosti[trida] = e_score * f_score
        
        # Normalizace
        total = sum(pravdepodobnosti.values())
        for k in pravdepodobnosti:
            pravdepodobnosti[k] /= total
        
        return pravdepodobnosti

# Test klasifikatoru
klasifikator = JednoduchyZvukovyKlasifikator()

# Vytvoreni testovacich signalu
t = np.linspace(0, 0.1, 800)

# Simulovane zvuky
test_zvuky = {
    'Ticho': np.random.randn(800) * 0.01,
    'Tlesknuti': np.exp(-50 * t) * np.sin(2 * np.pi * 300 * t),
    'Lusknuti': np.exp(-30 * t) * np.sin(2 * np.pi * 1000 * t) * 0.5,
    'Piskani': np.sin(2 * np.pi * 2000 * t) * 0.3
}

print("=" * 60)
print("SIMULACE KLASIFIKACE ZVUKU")
print("=" * 60)
print()

for nazev, signal in test_zvuky.items():
    predikce = klasifikator.predikuj(signal)
    nejlepsi = max(predikce, key=predikce.get)
    
    print(f"Zvuk: {nazev}")
    print(f"  Predikce: {nejlepsi.upper()}")
    print(f"  Pravdepodobnosti:")
    for trida, prob in sorted(predikce.items(), key=lambda x: -x[1]):
        bar = '█' * int(prob * 20)
        print(f"    {trida:12s} {bar:20s} {prob*100:.1f}%")
    print()

---

## 7. Export modelu z Teachable Machine

Po natrenini modelu v Teachable Machine ho muzete exportovat:

In [ ]:
# Postup exportu modelu

print("=" * 60)
print("JAK EXPORTOVAT MODEL Z TEACHABLE MACHINE")
print("=" * 60)
print()
print("1. TRENINK DOKONCEN")
print("   - Po treninku vidite Preview panel")
print("   - Otestujte model v realnem case")
print()
print("2. KLIK NA 'Export Model'")
print("   - Tlacitko vpravo nahore")
print()
print("3. VYBERTE FORMAT")
print("   - TensorFlow.js (pro web)")
print("   - TensorFlow (pro Python)")
print("   - TensorFlow Lite (pro mobily)")
print()
print("4. UPLOAD NEBO STAZENI")
print("   - 'Upload my model' -> ziskate URL")
print("   - 'Download' -> stahne ZIP soubor")
print()
print("5. POUZITI V APLIKACI")
print("   - Web: TensorFlow.js")
print("   - Python: TensorFlow/Keras")
print("   - Mobil: TensorFlow Lite")

In [ ]:
# Priklad pouziti exportovaneho modelu (pseudokod)

priklad_kod = '''
# Priklad pouziti audio modelu z Teachable Machine v Pythonu
# (Toto je ilustracni kod - potrebuje skutecny model)

import tensorflow as tf
import numpy as np

# Nacteni modelu
model = tf.keras.models.load_model('muj_audio_model')

# Tridy (v poradi jako v Teachable Machine)
tridy = ['pozadi', 'tlesknuti', 'lusknuti', 'piskani']

def klasifikuj_zvuk(audio_data):
    # Predikce
    predikce = model.predict(audio_data)
    
    # Nejlepsi trida
    nejlepsi_idx = np.argmax(predikce[0])
    nejlepsi_trida = tridy[nejlepsi_idx]
    jistota = predikce[0][nejlepsi_idx]
    
    return nejlepsi_trida, jistota

# Pouziti
trida, jistota = klasifikuj_zvuk(muj_zvuk)
print(f"Detekovan zvuk: {trida} (jistota: {jistota*100:.1f}%)")
'''

print("PRIKLAD KODU PRO POUZITI MODELU:")
print("=" * 40)
print(priklad_kod)

---

## 8. Mini-kviz

In [ ]:
# Mini-kviz
print("=" * 60)
print("KVIZ: AUDIO KLASIFIKACE A TEACHABLE MACHINE")
print("=" * 60)
print()

otazky = [
    {
        "otazka": "1. Proc je dulezite nahrat tridu 'pozadi' v Teachable Machine?",
        "moznosti": [
            "a) Protoze to vyzaduje system",
            "b) Aby model vedel, co ma ignorovat",
            "c) Pro zlepseni kvality zvuku",
            "d) Neni to potreba"
        ],
        "spravna": "b",
        "vysvetleni": "Trida pozadi uci model rozpoznat bezny sum a nereagovat na nej jako na udalost."
    },
    {
        "otazka": "2. Co je spektrogram?",
        "moznosti": [
            "a) Graf amplitudy v case",
            "b) 2D vizualizace frekvenci v case",
            "c) Typ mikrofonu",
            "d) Format zvukoveho souboru"
        ],
        "spravna": "b",
        "vysvetleni": "Spektrogram ukazuje, jake frekvence jsou pritomny v kazdem casovem okamziku - je to 'obrazek' zvuku."
    },
    {
        "otazka": "3. Co jsou MFCC?",
        "moznosti": [
            "a) Typ neuronove site",
            "b) Format audio souboru",
            "c) Priznaky zvuku zalozene na Mel skale",
            "d) Metoda komprese zvuku"
        ],
        "spravna": "c",
        "vysvetleni": "MFCC jsou nejpouzivanejsi priznaky pro rozpoznavani zvuku a reci."
    },
    {
        "otazka": "4. Proc pouzivame Mel skalu?",
        "moznosti": [
            "a) Je rychlejsi na vypocet",
            "b) Odpovida vnimani frekvence lidskym uchem",
            "c) Produkuje mensi soubory",
            "d) Je presnejsi nez FFT"
        ],
        "spravna": "b",
        "vysvetleni": "Mel skala modeluje nelinearni vnimani frekvence clovekem - nizke frekvence rozlisujeme lepe."
    },
    {
        "otazka": "5. Kolik vzorku by mela mit kazda trida v Teachable Machine?",
        "moznosti": [
            "a) 1-2 vzorky staci",
            "b) 8-10 a vice vzorku",
            "c) Presne 100 vzorku",
            "d) Na poctu nezalezi"
        ],
        "spravna": "b",
        "vysvetleni": "Vice vzorku = lepsi model. 8-10 je minimum pro zakladni funkcnost."
    }
]

for q in otazky:
    print(q["otazka"])
    for m in q["moznosti"]:
        print(f"   {m}")
    print(f"   Spravna odpoved: {q['spravna']}")
    print(f"   > {q['vysvetleni']}")
    print()

---

## 9. Shrnuti kapitoly

| Koncept | Popis |
|---------|-------|
| **Zvukovy signal** | Posloupnost cisel reprezentujici zvukove vlneni |
| **Vzorkovaci frekvence** | Kolikrat za sekundu merime zvuk (napr. 44100 Hz) |
| **Spektrogram** | 2D vizualizace frekvenci v case - "obrazek" zvuku |
| **MFCC** | Priznaky zvuku zalozene na Mel skale |
| **Mel skala** | Skala odpovidajici lidskemu vnimani frekvence |
| **Teachable Machine** | Nastroj pro trenink AI bez kodu |

### Klicove poznatky:
- AI "vidi" zvuk jako spektrogram nebo MFCC priznaky
- Trida "pozadi" je klicova pro spravnou funkci klasifikatoru
- Teachable Machine umoznuje rychle prototypovani bez kodu
- Principy klasifikace jsou stejne pro obrazky i zvuk

---

## 10. Vyzva pro vas

Zkuste tyto experimenty:

In [ ]:
# VYZVA 1: Vytvorte vlastni zvukovy klasifikator v Teachable Machine
print("VYZVA 1: Vytvorte zvukovy klasifikator")
print("="*40)
print()
print("Napady pro projekty:")
print("- Rozpoznavani 'ano' vs 'ne'")
print("- Detekce stekotu psa vs mnoukani kocky")
print("- Rozliseni hudebnych nastroju")
print("- Klasifikace typu hudby")
print()
print("Jdete na: https://teachablemachine.withgoogle.com/")

In [ ]:
# VYZVA 2: Experimentujte s ruznymi typy zvuku
print("VYZVA 2: Vytvorte a vizualizujte vlastni zvuky")
print("="*40)
print()
print("Zkuste zmenit parametry a sledovat spektrogramy:")
print("- Ruzne frekvence (100 Hz vs 1000 Hz vs 5000 Hz)")
print("- Kombinace vice frekvenci (akordy)")
print("- Ruzne tvary vln (sinus, obdelnik, pila)")
print("- Pridani sumu")

In [ ]:
# VYZVA 3: Prozkoumejte knihovnu librosa
print("VYZVA 3: Pouzijte librosa pro analyzu zvuku")
print("="*40)
print()
print("Knihovna librosa umi:")
print("- Nacitat audio soubory")
print("- Pocitat MFCC")
print("- Detekovat tempo a beat")
print("- Extrahovat chroma priznaky")
print()
print("Dokumentace: https://librosa.org/doc/")

---

## Dalsi kroky

V pristich kapitolach se naucime:
- Jak postavit skutecnou CNN pro obrazky
- Transfer learning - vyuziti predtrenovanych modelu
- Jak nasadit model do realne aplikace

**Audio klasifikace ma obrovske moznosti - od rozpoznavani reci az po detekci poruch stroju!**